In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

import requests
import json
import datetime as dt
from hdfs import InsecureClient
from json import dumps
import pandas as pd

/home/worker/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
from datetime import date, datetime
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt #그래프 패키지 모듈 등록
%matplotlib inline 
from databricks_api import DatabricksAPI
import requests
import json
import datetime as dt
from hdfs import InsecureClient
from pyspark.sql import SparkSession 
from pyspark.sql.functions import col, monotonically_increasing_id, row_number
from pyspark.sql.window import Window
from pyspark.sql.types import *

import csv
import sys
import datetime

ModuleNotFoundError: No module named 'databricks_api'

In [3]:
FILE_DIR='/hospital_code/'
FILE_NAME='/hospital_code/hospital_code2022-10-29.csv'
BASE_URL='https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&sq=&o=&q='


In [4]:
client = InsecureClient('http://localhost:9870', user='big')

In [5]:
from pyspark.sql import SparkSession 
spark = SparkSession \
.builder \
.master("local") \
.appName("Python Spark SQL basic example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()

In [6]:
hospital_code_csv = spark.read.csv('/hospital_code/hospital_code2022-10-29.csv',header=True,encoding='cp949')
hospital_code_csv.show()

+--------+------+----------+--------------+----------------+--------------------+--------------+----------+--------+------------+------------+----------+------------------+--------------+------------------------+--------------+--------------------+--------------------+--------------+--------+--------+------+------+--------------------+---------------------------------+------------+----------------+------------------+--------------+--------------+----------+--------+----------+------------+---------------------------------+---------------------------------+--------------+--------------+-------------+
|시군코드|시군명|인허가일자|인허가취소일자|영업상태구분코드|통합영업상태구분코드|통합영업상태명|영업상태명|폐업일자|휴업시작일자|휴업종료일자|재개업일자|소재지시설전화번호|소재지면적정보|                사업장명|업태구분명정보|             X좌표값|             Y좌표값|의료기관종별명|의료인수|입원실수|병상수|총면적|        진료과목내용|                 진료과목내용정보|지정취소일자|완화의료지정형태|완화의료담당부서명|특수구급차대수|일반구급차대수|총종업원수|구조사수|허가병상수|최초지정일자|                   소재지지번주소|                 소재지도로명주소|소재지우편번호|          경도|         위도|
+------

In [7]:
new_df=hospital_code_csv[hospital_code_csv['통합영업상태명']=='영업/정상']
tel=new_df.select('소재지시설전화번호')
tel_list=list(tel.toPandas()['소재지시설전화번호'])
type(tel_list)
new_df.show()

+--------+------+----------+--------------+----------------+--------------------+--------------+----------+--------+------------+------------+----------+------------------+--------------+------------------------+--------------+--------------------+--------------------+--------------+--------+--------+------+------+--------------------+---------------------------------+------------+----------------+------------------+--------------+--------------+----------+--------+----------+------------+---------------------------------+---------------------------------+--------------+--------------+-------------+
|시군코드|시군명|인허가일자|인허가취소일자|영업상태구분코드|통합영업상태구분코드|통합영업상태명|영업상태명|폐업일자|휴업시작일자|휴업종료일자|재개업일자|소재지시설전화번호|소재지면적정보|                사업장명|업태구분명정보|             X좌표값|             Y좌표값|의료기관종별명|의료인수|입원실수|병상수|총면적|        진료과목내용|                 진료과목내용정보|지정취소일자|완화의료지정형태|완화의료담당부서명|특수구급차대수|일반구급차대수|총종업원수|구조사수|허가병상수|최초지정일자|                   소재지지번주소|                 소재지도로명주소|소재지우편번호|          경도|         위도|
+------

In [8]:
new_df=new_df.withColumn('HOS_ID', row_number().over(Window.orderBy(monotonically_increasing_id())))

#전화번호  NULL값 제거,TEL컬럼을 PRIMARY KEY로 지정
new_df=new_df.select(col('HOS_ID'),
                    col('사업장명').alias('HOS_NAME'),
                     col('통합영업상태명').alias('STATUS'),
                     col('업태구분명정보').alias('HOS_TYPE'),
                     col('진료과목내용정보').alias('MEDI_COURSE'),
                     col('소재지도로명주소').alias('ADDR'),
                     col('소재지우편번호').alias('POST_CODE'),
                     col('경도').alias('LONGITUDE').cast('float'),
                     col('위도').alias('LATITUDE').cast('float'),
                     col('소재지시설전화번호').alias('TEL'),
                     col('시군코드').alias('SIDO_ID')
                    )
# X좌표값, Y좌표값, 경도, 구조사수, 병상수, 사업장명, 소재지도로명주소, 소재지면적정보, 
# 소재지시설전화번호, 소재지우편번호, 소재지지번주소, 시군명, 시군코드, 업태구분명정보, 
# 영업상태구분코드, 영업상태명, 완화의료담당부서명, 완화의료지정형태, 위도, 의료기관종별명, 
# 의료인수, 인허가일자, 인허가취소일자, 일반구급차대수, 입원실수, 재개업일자, 지정취소일자, 
# 진료과목내용, 진료과목내용정보, 총면적, 
# 총종업원수, 최초지정일자, 통합영업상태구분코드, 통합영업상태명, 특수구급차대수, 폐업일자, 허가병상수, 휴업시작일자, 휴업종료일자

new_df.show()

+------+------------------------+---------+--------+---------------------------------+---------------------------------+---------+----------+---------+---------------+-------+
|HOS_ID|                HOS_NAME|   STATUS|HOS_TYPE|                      MEDI_COURSE|                             ADDR|POST_CODE| LONGITUDE| LATITUDE|            TEL|SIDO_ID|
+------+------------------------+---------+--------+---------------------------------+---------------------------------+---------+----------+---------+---------------+-------+
|     1|          연세림치과의원|영업/정상|치과의원|구강악안면외과, 치과보철과, 치...| 경기도 수원시 영통구 효원로 3...|    16545| 127.04393|37.259243|   031-217-7712|  41110|
|     2| 환한미소 바른이치과의원|영업/정상|치과의원|구강악안면외과, 치과보철과, 치...|경기도 수원시 영통구 중부대로2...|    16531| 127.04383|37.274403|  031-8013-7575|  41110|
|     3|         영 연세치과의원|영업/정상|치과의원|구강악안면외과, 치과보철과, 치...| 경기도 수원시 영통구 봉영로17...|    16713|127.084175| 37.26844|   031-203-1793|  41110|
|     4|    큰나무가정의학과의원|영업/정상|    의원|  내과, 외과, 정형외과, 소아청...| 경기도 수원시 영

In [9]:
#다음에 전화번호로 병원 검색->카카오맵 접속 링크 구하기
kakao_url=[]
for i in tel_list:
    daum_res=requests.get(BASE_URL+i)
    d_soup=BeautifulSoup(daum_res.content,'lxml')
    try:
        s1=d_soup.find('div',{'class':'pannel'}).find('li')
        link=s1.find('a')['href']
        #문자열+숫자 중에서 숫자만 빼오기
        numbers = re.sub(r'[^0-9]', '', link)
        #카카오 url 추출
        kakao_url.append('https://place.map.kakao.com/m/main/v/'+numbers)
        
    except:
        pass
kakao_url

TypeError: can only concatenate str (not "NoneType") to str

In [26]:
#카카오맵 접속링크로 하나씩 접속해서 정보 크롤링

hospital_name_arr=[]
phonenum_arr=[]
sigungu_arr=[]

depart_arr=[]
feedback_arr=[]
review_txt_arr=[]

for i in kakao_url[:5]:
    kakao_res=requests.get(i)
    k_json=kakao_res.json()
    
    open_cnt=len(k_json['hospitalInfo']['list'][1])
#     print(k_json['hospitalInfo']['list'][1]['openHourList'])

#     #병원명,전화번호,시군구정보
#     hospital_name=k_json['basicInfo']['placenamefull']
#     hospital_name_arr.append(hospital_name)
#     phonenum=k_json['basicInfo']['phonenum']
#     phonenum_arr.append(phonenum)
#     sigungu=k_json['basicInfo']['address']['region']['newaddrfullname']
#     sigungu_arr.append(sigungu)
    
#     #진료과목
#     cnt=len(k_json['hospitalInfo']['list'][0]['subjectList'])
#     depart_in_arr=[]
#     for k in range(cnt):
#         depart_in_arr.append(k_json['hospitalInfo']['list'][0]['subjectList'][k]['name'])
#     depart_arr.append(depart_in_arr)
        
#     #별점
#     try:
#         feedback=k_json['basicInfo']['feedback']['scoresum']/k_json['basicInfo']['feedback']['scorecnt']
#         feedback_arr.append(feedback)
#     except:
#         feedback_arr.append(0)
    
#     #리뷰 텍스트
#     try:
#         r_cnt=len(k_json['comment']['list'])
#         review_txt_in_arr=[]
#         for j in range(r_cnt) :
#             review_txt_in_arr.append(k_json['comment']['list'][j]['contents'])
#         review_txt_arr.append(review_txt_in_arr)
#     except:
#         review_txt_arr.append('-')


In [27]:
#영업시간
openhour_arr=[]

openhour_arr.append(k_json['basicInfo']['openHour']['periodList'][0]['timeList'])
    
openhour_arr

# openhour_arr.append(k_json['basicInfo']['openHour']['periodList'][0]['timeList'])

KeyError: 'openHour'

In [37]:
k_json['hospitalInfo']['list'][0]['infoList'][1]['desc2']


'일반의 1명'

In [48]:
df=pd.DataFrame(zip(hospital_name_arr,review_txt_arr,sigungu_arr,depart_arr,feedback_arr,review_txt_arr), \
             columns=['병원명','전화번호','시군구','진료과목','별점','리뷰텍스트'])

df

,병원명,전화번호,시군구,진료과목,별점,리뷰텍스트
0,365스마트의원,[코로나 검사 때문에 방문했는데 간호사 분 중 안경쓰신 분 굉장히 신경질적이고 불친...,경기 수원시 장안구,"[내과, 소아청소년과, 이비인후과, 피부과, 가정의학과]",2.625000,[코로나 검사 때문에 방문했는데 간호사 분 중 안경쓰신 분 굉장히 신경질적이고 불친...
1,연세림치과의원,[몇 년 째 진료 받는 치과예요. 이가 시려서 스케일링 힘들다고 말씀드렸더니 세심하...,경기 수원시 영통구,"[치과, 구강악안면외과, 치과보철과, 치과교정과, 소아치과, 치주과, 치과보존과, ...",4.000000,[몇 년 째 진료 받는 치과예요. 이가 시려서 스케일링 힘들다고 말씀드렸더니 세심하...
2,환한미소 바른이치과의원,[저는 완전 심각한 입툭튀에 삐뚤빼뚤한 치아였는데 완벽하게 들어가고 가지런해졌어요!...,경기 수원시 영통구,"[치과, 구강악안면외과, 치과보철과, 치과교정과, 소아치과, 치주과, 치과보존과, ...",5.000000,[저는 완전 심각한 입툭튀에 삐뚤빼뚤한 치아였는데 완벽하게 들어가고 가지런해졌어요!...
3,영연세치과의원,"[진료 잘 봤습니다 신뢰감이 있어 좋았고 환자에게 설명도 잘 해주셨네요, 친절하게 ...",경기 수원시 영통구,"[치과, 구강악안면외과, 치과보철과, 치과교정과, 소아치과, 치주과, 치과보존과, ...",5.000000,"[진료 잘 봤습니다 신뢰감이 있어 좋았고 환자에게 설명도 잘 해주셨네요, 친절하게 ..."
4,큰나무가정의학과,"[항상 자세히 설명 해주는 병원 개원이후 지금까지, 간호사가 정말 불친절 합니다.\...",경기 수원시 영통구,"[내과, 외과, 정형외과, 소아청소년과, 이비인후과, 피부과, 비뇨의학과, 가정의학과]",3.400000,"[항상 자세히 설명 해주는 병원 개원이후 지금까지, 간호사가 정말 불친절 합니다.\..."
...,...,...,...,...,...,...
66,서울쎈정형외과의원,"[의사말투 깡패같고, 싸가지 없음.\n전체적으로 불친절함, 오십견으로치료받았는데좋아...",경기 수원시 팔달구,"[정형외과, 신경외과, 마취통증의학과, 재활의학과]",2.800000,"[의사말투 깡패같고, 싸가지 없음.\n전체적으로 불친절함, 오십견으로치료받았는데좋아..."
67,다정정신건강의학과의원,[의사선생님과 간호사선생님분들 모두 친절하십니다\n 불친절 하다는 말이 있길래 약간...,경기 수원시 권선구,[정신건강의학과],4.000000,[의사선생님과 간호사선생님분들 모두 친절하십니다\n 불친절 하다는 말이 있길래 약간...
68,스마일이비인후과의원,[의사가 친절하기는 한데 귀가 다쳐서 치료를 했어요.\n완치가 되지 않았어요.\n치...,경기 수원시 권선구,"[내과, 소아청소년과, 이비인후과, 피부과]",4.000000,[의사가 친절하기는 한데 귀가 다쳐서 치료를 했어요.\n완치가 되지 않았어요.\n치...
69,연세앤치과의원,"[괜찮은데 예전에 다니던곳보단 별로였어요, 치료 잘받았습니다. 별한개도 아까움, 여...",경기 수원시 영통구,"[구강악안면외과, 치과보철과, 치과교정과, 소아치과, 치주과, 치과보존과, 구강내과...",4.142857,"[괜찮은데 예전에 다니던곳보단 별로였어요, 치료 잘받았습니다. 별한개도 아까움, 여..."


## 병원데이터 콛,

In [ ]:
FILE_DIR='/hospital_code/'
FILE_NAME='/hospital_code/hospital_code2022-10-29.csv'
BASE_URL='https://search.daum.net/search?w=tot&DA=YZR&t__nil_searchbox=btn&sug=&sugo=&sq=&o=&q='

client = InsecureClient('http://localhost:9870', user='big')

from pyspark.sql import SparkSession 
spark = SparkSession \
.builder \
.master("local") \
.appName("Python Spark SQL basic example") \
.config("spark.some.config.option", "some-value") \
.getOrCreate()



In [ ]:
hospital_code_csv = spark.read.csv('/hospital_code/hospital_code2022-10-18.csv',header=True,encoding='cp949')
hospital_code_csv.show()